<a href="https://colab.research.google.com/github/vkthakur90/AI_Applications/blob/master/Valuation_of_Stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
rate = np.log(1 + 7/100)
life = 15
paths = 100000

In [3]:
infy_df = pd.DataFrame({
    'Year' : 2013 + np.arange(12),
    'CFO' : [7373,	9825,	8353,	10028,	11531,	13218,	14841,	17003,	23224,	23885,	22467,	25210]
})

In [4]:
infy_df

,Year,CFO
0,2013,7373
1,2014,9825
2,2015,8353
3,2016,10028
4,2017,11531
5,2018,13218
6,2019,14841
7,2020,17003
8,2021,23224
9,2022,23885


In [5]:
infy_df.eval('S = log(CFO)', inplace = True)
infy_df['prev_S'] = infy_df['S'].shift(1)

In [6]:
infy_df.dropna(inplace = True)

In [7]:
infy_df.eval('r = S - prev_S', inplace = True)

In [8]:
infy_df

,Year,CFO,S,prev_S,r
1,2014,9825,9.192685,8.905580,0.287105
2,2015,8353,9.030376,9.192685,-0.162309
3,2016,10028,9.213136,9.030376,0.182760
4,2017,11531,9.352794,9.213136,0.139658
5,2018,13218,9.489335,9.352794,0.136540
6,2019,14841,9.605149,9.489335,0.115814
7,2020,17003,9.741145,9.605149,0.135996
8,2021,23224,10.052942,9.741145,0.311796
9,2022,23885,10.081006,10.052942,0.028064
10,2023,22467,10.019803,10.081006,-0.061203


In [9]:
mu = infy_df['r'].mean()
sigma = infy_df['r'].std()

In [10]:
(mu, sigma)

(0.1117650965629378, 0.13778420163804303)

In [11]:
W = np.cumsum(np.random.standard_normal(size = [life, paths]), axis = 0)
t = np.arange(life)[..., np.newaxis]

forecast_data = np.exp((mu - rate) * t + sigma * W)

In [12]:
forecast_df = pd.DataFrame(
    data = forecast_data,
    columns = [f'DCF_{idx + 1}' for idx in range(paths)]
)

In [13]:
pe_ratio = forecast_df.sum().mean()

In [14]:
pe_ratio

22.612951689664218

In [15]:
forecast_df

,DCF_1,DCF_2,DCF_3,DCF_4,DCF_5,DCF_6,DCF_7,DCF_8,DCF_9,DCF_10,...,DCF_99991,DCF_99992,DCF_99993,DCF_99994,DCF_99995,DCF_99996,DCF_99997,DCF_99998,DCF_99999,DCF_100000
0,1.092019,1.011451,1.213031,1.118738,1.011787,0.973123,1.195172,0.976318,0.981868,0.969511,...,1.183494,0.927157,1.097669,1.047827,0.814346,0.965573,1.093737,1.018969,1.018885,1.105768
1,1.243107,0.951740,1.322055,1.194689,0.970060,0.752231,1.244490,1.358380,0.890651,0.848753,...,1.418386,1.146649,1.178583,0.914191,0.803376,0.827127,1.084293,1.122766,1.062053,0.886669
2,1.351796,1.100095,1.424534,1.170412,0.959243,0.935308,0.992013,1.708660,1.185646,0.854706,...,1.543391,1.208898,1.759744,0.837919,0.791217,1.001767,1.107060,1.291121,0.735404,0.843128
3,1.147628,1.046924,1.417846,1.273987,1.170876,1.054401,0.935779,1.786760,1.052724,0.905421,...,1.498520,1.199539,1.957247,0.798163,0.722630,1.024794,1.262068,1.355513,0.834494,0.927168
4,1.186627,1.063698,1.444373,1.748577,1.233813,1.664799,0.818181,2.140799,1.052061,1.022423,...,1.434457,1.153673,1.835407,0.914684,0.600312,1.026158,1.334563,1.337256,0.951985,0.844321
5,1.554408,1.068370,1.735932,1.850398,1.195653,1.735658,0.731366,2.252578,1.101757,1.042440,...,1.659006,1.271820,2.262015,1.025130,0.625683,1.154060,1.657495,1.674688,0.880729,1.003960
6,1.900004,0.892181,1.548375,2.309531,1.015193,1.520889,0.648143,2.704343,0.953338,1.190424,...,1.660852,1.435617,3.171527,1.194957,0.525079,1.339949,1.738445,1.411735,0.803914,1.151477
7,1.915324,0.762281,1.501677,2.605726,1.144918,1.514955,0.599970,3.513890,1.085325,1.210017,...,1.461596,1.217160,3.052756,1.221084,0.503761,1.345898,1.796576,1.179289,0.994063,1.259214
8,1.750679,0.789357,2.221361,2.706546,1.370145,1.437482,0.806989,3.766847,1.261593,1.294606,...,1.383050,1.260403,2.886543,1.093304,0.665009,1.258692,1.600829,1.221617,0.850253,1.412618
9,2.038815,0.938860,2.113404,2.364056,1.630886,1.136531,0.901776,2.718337,1.252130,1.274169,...,1.325167,1.355583,2.945128,1.128091,0.974034,1.224804,1.982896,0.939484,0.794196,1.376370
